In [103]:
import pandas as pd
import numpy as np

In [90]:
# load
loan_result=pd.read_csv("../data/loan_result.csv")
log_data=pd.read_csv("../data/log_data.csv")
user_spec=pd.read_csv("../data/user_spec.csv")

# columns localize
loan_result.columns=['신청서_번호', '한도조회_일시', '금융사_번호', '상품_번호', '승인한도', '승인금리', '신청_여부(y)']
log_data.columns=['유저_번호', '행동명', '행동일시', '운영체제', '앱_버전', '일_코드']
user_spec.columns=['신청서_번호', '유저_번호', '유저_생년월일', '유저_성별', '생성일시', '한도조회_당시_유저_신용점수', '연소득', '근로형태', '입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출_목적', '개인회생자_여부', '개인회생자_납입_완료_여부', '기대출수', '기대출금액'] # 건강보험 타입이 정의서에는 있는데 데이터엔 없는듯

In [91]:
log_data.shape

(17843993, 6)

In [92]:
# log_data sort by time(asc)
log_data=log_data.sort_values('행동일시').reset_index(drop=True)
log_data.head()

,유저_번호,행동명,행동일시,운영체제,앱_버전,일_코드
0,125129,UseLoanManage,2022-03-01 00:00:00,Android,3.8.0,2022-03-01
1,385811,Login,2022-03-01 00:00:00,Android,3.8.0,2022-03-01
2,720911,OpenApp,2022-03-01 00:00:00,iOS,3.6.0,2022-03-01
3,706858,UseLoanManage,2022-03-01 00:00:03,Android,3.8.0,2022-03-01
4,512341,EndLoanApply,2022-03-01 00:00:03,Android,3.8.0,2022-03-01


In [93]:
log_data['행동일시'] = pd.to_datetime(log_data['행동일시'])
log_data['일_코드'] = pd.to_datetime(log_data['일_코드'])
loan_result['한도조회_일시'] = pd.to_datetime(loan_result['한도조회_일시'])
user_spec['생성일시'] = pd.to_datetime(user_spec['생성일시'])

In [119]:
users=set(user_spec['유저_번호'])
log_user=set(log_data['유저_번호'])
count=[]

for num in log_user:
    if num in users:
        count.append(num)

print(len(count))

370436


In [127]:
from tqdm import tqdm

user_df=user_spec.copy()
log_df=log_data.copy()

# 각 13시간 정도...

for i in tqdm(range(len(user_spec))):
    if user_spec['유저_번호'].iloc[i] not in count:
        user_df.drop([i])



  0%|          | 24778/17843993 [41:17<494:59:15, 10.00it/s] 


KeyboardInterrupt: 

In [130]:
len(user_df), len(user_spec)

(1394216, 1394216)

In [129]:
user_df.to_csv("./user_df.csv", index=False)

In [145]:
uid=pd.DataFrame(count, columns=['유저_번호']).copy()
user_df2=pd.merge(left=log_data, right=uid, how="inner", on="유저_번호")
user_df2.head()

,유저_번호,행동명,행동일시,운영체제,앱_버전,일_코드
0,125129,UseLoanManage,2022-03-01 00:00:00,Android,3.8.0,2022-03-01
1,125129,OpenApp,2022-03-02 01:53:05,Android,3.8.0,2022-03-02
2,125129,Login,2022-03-02 01:53:12,Android,3.8.0,2022-03-02
3,125129,StartLoanApply,2022-03-02 01:53:18,Android,3.8.0,2022-03-02
4,125129,ViewLoanApplyIntro,2022-03-02 01:53:19,Android,3.8.0,2022-03-02


In [146]:
len(user_df2), len(log_data), len(set(log_data['유저_번호']))

(16131586, 17843993, 584636)

In [147]:
user_df2.to_csv("./user_df2.csv", index=False)

In [ ]:
for i in tqdm(range(len(log_data))):
    if log_data['유저_번호'].iloc[i] not in count:
        log_df.drop([i])

In [ ]:
len(user_df), len(log_df), len(user_spec), len(log_data)

# 추가변수

## (1) user_spec에 추가
- log_data -> user_spec

### 1. 사용자 관련
1) 신규 사용자
2) 재방문 사용자 - 재방문까지의 시간 차
- 재방문 회수가 여러번이면 평균값
5) 사용자별 방문 수 = 세션 수
7) 사용자가 방문한 시간대 접속자 수

### 2. 화면 - 유저 기준
1) 방문 당 스크린 뷰(스크린뷰/세션)
2) 방문 당 유니크 뷰(유니크뷰/세션)

### 3. 경로 - 유저 기준
1) 개별 경로
2) 이동수
3) 평균 이동수
4) 동일 경로 반복 이동

### 4. 액션 - 유저 기준
1) 방문 당 액션 = 액션수/세션수
2) 무반응 액션율

### 5. 그 외
..

#### 신청서_번호 당
- 

#### 유저_번호 당
- 

#### 신경써야할 부분
- 신청서를 특정 기간을 두고 여러번 낸 유저의 경우, 처음 낸 신청서 때랑 나중에 낸 신청서 때를 구분하는 게 맞나?!
- 근데 로그 데이터에는 신청서 번호가 없음

---

#### 행동별 회수
- 전체 event

In [94]:

open_app=log_data[log_data['행동명']=="OpenApp"]
open_app=open_app.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
open_app.rename(columns={'행동명':'앱실행'}, inplace=True)

open_app2=log_data[log_data['행동명']=="SignUp"]
open_app2=open_app2.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
open_app2.rename(columns={'행동명':'회원가입'}, inplace=True)

open_app3=log_data[log_data['행동명']=="Login"]
open_app3=open_app3.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
open_app3.rename(columns={'행동명':'로그인'}, inplace=True)

start_loan=log_data[log_data['행동명']=="ViewLoanApplyIntro"]
start_loan=start_loan.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
start_loan.rename(columns={'행동명':'인트로_조회'}, inplace=True)

use_loan_manage=log_data[log_data['행동명']=="StartLoanApply"]
use_loan_manage=use_loan_manage.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
use_loan_manage.rename(columns={'행동명':'한도조회_시작'}, inplace=True)

use_loan_manage2=log_data[log_data['행동명']=="CompleteIDCertification"]
use_loan_manage2=use_loan_manage2.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
use_loan_manage2.rename(columns={'행동명':'본인인증완료'}, inplace=True)

use_loan_manage3=log_data[log_data['행동명']=="EndLoanApply"]
use_loan_manage3=use_loan_manage3.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
use_loan_manage3.rename(columns={'행동명':'대출관리_회수'}, inplace=True)

use_loan_manage4=log_data[log_data['행동명']=="UseLoanManage"]
use_loan_manage4=use_loan_manage4.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
use_loan_manage4.rename(columns={'행동명':'대출관리_회수'}, inplace=True)

use_prepay_calc=log_data[log_data['행동명']=="UsePrepayCalc"]
use_prepay_calc=use_prepay_calc.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
use_prepay_calc.rename(columns={'행동명':'여윳돈_회수'}, inplace=True)

use_dsr_calc=log_data[log_data['행동명']=="UseDSRCalc"]
use_dsr_calc=use_dsr_calc.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
use_dsr_calc.rename(columns={'행동명':'DSR_회수'}, inplace=True)

get_credit=log_data[log_data['행동명']=="GetCreditInfo"]
get_credit=get_credit.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
get_credit.rename(columns={'행동명':'KCB_회수'}, inplace=True)



In [99]:
# 없으면 NaN
user_spec_1=pd.merge(left=user_spec, right=open_app, how="left", on="유저_번호")
user_spec_2=pd.merge(left=user_spec_1, right=start_loan, how="left", on="유저_번호")
user_spec_3=pd.merge(left=user_spec_2, right=use_loan_manage, how="left", on="유저_번호")
user_spec_4=pd.merge(left=user_spec_3, right=use_prepay_calc, how="left", on="유저_번호")
user_spec_5=pd.merge(left=user_spec_4, right=use_dsr_calc, how="left", on="유저_번호")
user_spec_6=pd.merge(left=user_spec_5, right=open_app2, how="left", on="유저_번호")
user_spec_7=pd.merge(left=user_spec_6, right=open_app3, how="left", on="유저_번호")
user_spec_8=pd.merge(left=user_spec_7, right=use_loan_manage2, how="left", on="유저_번호")
user_spec_9=pd.merge(left=user_spec_8, right=use_loan_manage3, how="left", on="유저_번호")
user_spec_10=pd.merge(left=user_spec_9, right=use_loan_manage4, how="left", on="유저_번호")
user_spec_add_counts=pd.merge(left=user_spec_10, right=get_credit, how="left", on="유저_번호")
# user_spec_add_counts['총_이벤트수']=
user_spec_add_counts.head(5)

# fillna하고 싶은데, 애초에 접속을 아예 안 한 사람들도 있을 것 같아서 좀 어려움!

,신청서_번호,유저_번호,유저_생년월일,유저_성별,생성일시,한도조회_당시_유저_신용점수,연소득,근로형태,입사연월,고용형태,...,인트로_조회,한도조회_시작,여윳돈_회수,DSR_회수,회원가입,로그인,본인인증완료,한도조회_결과,대출관리_회수,KCB_회수
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,...,3.0,3.0,NaN,NaN,NaN,5.0,3.0,3.0,2.0,7.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,...,2.0,3.0,NaN,NaN,NaN,3.0,2.0,5.0,3.0,3.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,...,8.0,7.0,NaN,NaN,NaN,8.0,6.0,8.0,3.0,5.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,...,14.0,11.0,NaN,NaN,NaN,NaN,10.0,13.0,18.0,12.0


#### 세션 당 스크린수

In [106]:
# 앱실행(세션) 수만 fillna
user_spec_add_counts.loc[user_spec_add_counts['앱실행'] != user_spec_add_counts['앱실행'], '앱실행'] = 0.0

user_spec_add_counts['앱실행당_한도조회']=user_spec_add_counts['한도조회_시작']/user_spec_add_counts['앱실행']
user_spec_add_counts['앱실행당_대출관리']=user_spec_add_counts['대출관리_회수']/user_spec_add_counts['앱실행']
user_spec_add_counts['앱실행당_여윳돈']=user_spec_add_counts['여윳돈_회수']/user_spec_add_counts['앱실행']
user_spec_add_counts['앱실행당_DSR']=user_spec_add_counts['DSR_회수']/user_spec_add_counts['앱실행']
user_spec_add_counts['앱실행당_KCB']=user_spec_add_counts['KCB_회수']/user_spec_add_counts['앱실행']

user_spec_add_counts=user_spec_add_counts.replace([np.inf, -np.inf], np.nan)

user_spec_add_counts.head(3)

,신청서_번호,유저_번호,유저_생년월일,유저_성별,생성일시,한도조회_당시_유저_신용점수,연소득,근로형태,입사연월,고용형태,...,로그인,본인인증완료,한도조회_결과,대출관리_회수,KCB_회수,앱실행당_한도조회,앱실행당_대출관리,앱실행당_여윳돈,앱실행당_DSR,앱실행당_KCB
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,...,5.0,3.0,3.0,2.0,7.0,0.75,0.5,NaN,NaN,1.75
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,...,3.0,2.0,5.0,3.0,3.0,1.50,1.5,NaN,NaN,1.50


### 문제 있는 것들
- Log df에서 한 줄 씩 읽어야하는 것들
- 신청서 작성 전 방문, 로그인, 한도조회, 대출관리, 여윳돈, dsr, kcb 회수 

#### 신규 방문, 재방문(로그인 기준)

In [116]:
user_enter=user_spec_add_counts[['신청서_번호', '유저_번호', '로그인']]
user_enter['진행회수']=0

login_app=log_data[log_data['행동명']=="Login"]
# login_app=login_app.groupby(['유저_번호'], as_index=False).count()[['유저_번호','행동명']]
# login_app.rename(columns={'행동명':'로그인_회수'}, inplace=True)

for i in range(len(login_app)):
    if i%1000000==0: print(i)
    for j in range(len(user_enter)):
        if user_enter['유저_번호'].iloc[j]==login_app['유저_번호'].iloc[i]:
            user_enter['진행회수'].iloc[j]+=1
            continue
user_enter.head()

/Users/nkeum/opt/anaconda3/envs/smartfarm/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0


/Users/nkeum/opt/anaconda3/envs/smartfarm/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


KeyboardInterrupt: 

In [ ]:
# 1) 신규 사용자 + 재방문 사용자 = 신청서 작성 전 방문 회수, 로그인 회수, 
# - 기준을 앱실행(OpenApp), 회원가입(SignUp), 로그인(Login)?
# 시간이 너무 오래걸려서, 거를 거 거르고 돌리려구용
from datetime import datetime
from tqdm import tqdm

user_spec['작성_전_방문_회수']=-1


for i in tqdm(range(len(user_spec))):
    aid=user_spec['신청서_번호'].iloc[i]
    uid=user_spec['유저_번호'].iloc[i]
    created_time=user_spec['생성일시'].iloc[i]
    
    # log_data 중 유저 번호가 같고, 행동이 OpenApp인 df
    uid_log_df=log_data[(log_data['유저_번호']==uid)&(log_data['행동명']=='OpenApp')]
    
    count=0
    for j in range(len(uid_log_df)):
        # 신청서를 작성한 시기보다 이전에 OpenApp한 회수
        if uid_log_df['행동일시'].iloc[j] < created_time:
            count+=1
    user_spec['작성_전_방문_회수'].iloc[i]=count


In [ ]:
user_spec['작성_전_방문_회수'].describe()

count    1394216.0
mean          -1.0
std            0.0
min           -1.0
25%           -1.0
50%           -1.0
75%           -1.0
max           -1.0
Name: 작성_전_방문_회수, dtype: float64

In [ ]:
user_spec['작성_전_로그인_회수']=-1
user_spec['작성_전_한도조회_인트로_회수']=-1
user_spec['작성_전_대출관리_회수']=-1
user_spec['작성_전_여윳돈_회수']=-1
user_spec['작성_전_DSR_회수']=-1
user_spec['작성_전_KCB_회수']=-1

event_list=['Login', 'ViewLoanApplyIntro', 'UseLoanManage', 'UserPrepayCalc', 'UserDSRCalc', 'GetCreditInfo']
count_list=['작성_전_로그인_회수', '작성_전_한도조회_인트로_회수', '작성_전_대출관리_회수', '작성_전_여윳돈_회수', '작성_전_DSR_회수', '작성_전_KCB_회수']

for i in tqdm(range(len(user_spec))):
    aid=user_spec['신청서_번호'].iloc[i]
    uid=user_spec['유저_번호'].iloc[i]
    created_time=user_spec['생성일시'].iloc[i]
    
    
    # log_data 중 유저 번호가 같고, 행동이 k인 df
    for k in range(len(count_list)):
        uid_log_df=log_data[(log_data['유저_번호']==uid)&(log_data['행동명']==event_list[k])]
        
        count=0
        for j in range(len(uid_log_df)):
            # 신청서를 작성한 시기보다 이전에 OpenApp한 회수
            if uid_log_df['행동일시'].iloc[j] < created_time:
                count+=1
        user_spec[count_list[k]].iloc[i]=count

In [83]:
log_data[log_data['유저_번호']==118218]

,유저_번호,행동명,행동일시,운영체제,앱_버전,일_코드
305199,118218,GetCreditInfo,2022-03-02 19:44:40,android,447,2022-03-02
3534618,118218,GetCreditInfo,2022-03-28 11:09:28,android,451,2022-03-28
6422224,118218,GetCreditInfo,2022-04-18 11:04:06,android,460,2022-04-18
16490174,118218,GetCreditInfo,2022-06-23 14:39:20,android,469,2022-06-23


In [ ]:
user_spec[count_list].describe()

In [24]:
len(log_data[log_data['행동명']=='SignUp']), len(set(log_data['유저_번호'])), len(log_data)

(34892, 584636, 17843993)

In [ ]:
# 5) 사용자별 방문 수 = 세션 수


In [ ]:
# 7) 사용자가 방문한 시간대 접속자 수

## 주요 경로
### 1. 경로의 종류
#### 1) 5개의 스크린
- 한도조회시작(StartLoanApply), 대출관리서비스 이용(UseLoanManage), 여윳돈 계산기 이용(UsePrepayCalc), DSR 계산기 이용(UseDSRCalc), KCB 신용정보 조회(GetCreditInfo)
- ['StartLoanApply', 'UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditInfo']
#### 2) 시작점-종료점
- OpenApp부터 그 다음 OpenApp 까지

### 2. 슬라이싱
- 0. log_data에서 5개의 스크린만 뽑기
- 1. log_data를 user별로 
- 2. OpenApp 앞뒤를 쪼개서 여러개의 리스트로 합친
- 3. 전체 route_list에 넣음
- 4. 겹치는 것의 종류 확인
    + 5!+4!+3!+2!+1!=153 보다 많다면 쪼개야하나.. 일단 최대 길이 찾아서 대기..
- 5. value_counts()

In [165]:
screens=['StartLoanApply', 'UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditInfo']
main_route_log_df=log_data[(log_data['행동명']=="OpenApp")|
                           (log_data['행동명']==screens[0])|
                           (log_data['행동명']==screens[1])|
                           (log_data['행동명']==screens[2])|
                           (log_data['행동명']==screens[3])|
                           (log_data['행동명']==screens[4])
                           ]
main_route_log_df.reset_index(drop=True, inplace=True)
len(main_route_log_df), len(log_data), len(set(main_route_log_df['유저_번호']))

(9587604, 17843993, 574778)

In [ ]:
users=list(set(main_route_log_df['유저_번호']))
route_list=[]

# target의 idx들(복수)를 list로 추출
def list_idx(arr, target):
    return [i for i in range(len(arr)) if arr[i]==target]

# 유저별로 OpenApp 기준 슬라이싱
for i in tqdm(range(len(users))):
    df=main_route_log_df[main_route_log_df['유저_번호']==users[i]]
    idx_list=list_idx(list(df['행동명']), 'OpenApp')
    if len(idx_list)>0 and len(df)>0:
        for i in range(len(idx_list)):
            start=idx_list[i]
            try:
                end=idx_list[i+1]
            except:
                end=len(df)
            route_list.append(list(df['행동명'].iloc[start:end]))
        # route_list.append([df['행동명'].iloc[idx_list[i]:min(idx_list[i+1], len(df))] for i in range(len(idx_list))])
    else: continue

In [179]:
len(route_list)

3460762

In [192]:
route_set = []

for i in tqdm(range(len(route_list))):
    if route_list[i] not in route_set:
        route_set.append(route_list[i])

route_set.sort()
len(route_set)

100%|██████████| 3460762/3460762 [00:13<00:00, 260638.53it/s]


24883

In [193]:
len(route_set), route_set[-1]

(24883, ['OpenApp', 'UsePrepayCalc', 'UseDSRCalc'])

In [194]:
route_set[:5]


[['OpenApp'],
 ['OpenApp', 'GetCreditInfo'],
 ['OpenApp', 'GetCreditInfo', 'GetCreditInfo'],
 ['OpenApp', 'GetCreditInfo', 'GetCreditInfo', 'GetCreditInfo'],
 ['OpenApp',
  'GetCreditInfo',
  'GetCreditInfo',
  'GetCreditInfo',
  'GetCreditInfo']]

In [185]:
np.save('./route_set', np.array(route_set))
np.save('./route_set.npy', np.array(route_set))

In [398]:
from asyncore import poll3
from itertools import permutations
from itertools import combinations

log_init=[('StartLoanApply'), ('UseLoanManage'), ('UsePrepayCalc'), ('UseDSRCalc'), ('GetCreditInfo')]
log_columns=[('StartLoanApply'), ('UseLoanManage'), ('UsePrepayCalc'), ('UseDSRCalc'), ('GetCreditInfo')]

p2=list(permutations(log_init, 2))
p3=list(permutations(log_init, 3))
p4=list(permutations(log_init, 4))
p5=list(permutations(log_init, 5))

n1=5
n2=len(p2)
n3=len(p3)
n4=len(p4)
n5=len(p5)

for arr in p2:
    log_columns.append(arr)
for arr in p3:
    log_columns.append(arr)
for arr in p4:
    log_columns.append(arr)
for arr in p5:
    log_columns.append(arr)
    
log_columns.append((''))

len(log_columns), n1+n2+n3+n4+n5+1

(326, 326)

In [399]:
log_ngram_df=pd.DataFrame([0]*len(log_columns), index=log_columns, columns=['count'])

count=0
for i in range(len(route_list)):
    if len(route_list[i])==1: count+=1
log_ngram_df.loc['']=count

log_ngram_df

,count
StartLoanApply,0
UseLoanManage,0
UsePrepayCalc,0
UseDSRCalc,0
GetCreditInfo,0
...,...
"(GetCreditInfo, UseDSRCalc, UseLoanManage, StartLoanApply, UsePrepayCalc)",0
"(GetCreditInfo, UseDSRCalc, UseLoanManage, UsePrepayCalc, StartLoanApply)",0
"(GetCreditInfo, UseDSRCalc, UsePrepayCalc, StartLoanApply, UseLoanManage)",0
"(GetCreditInfo, UseDSRCalc, UsePrepayCalc, UseLoanManage, StartLoanApply)",0


In [401]:
# screens=['StartLoanApply', 'UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditInfo']

for i in tqdm(range(len(route_list))):
    if len(route_list[i])==1: continue
    for n in range(1, len(route_list[i])):
        # 원소 안의 2~5개의 순서 지킨 조합에서
            # range에서도 1 빠지고 슬라이싱에서도 1 빠져서 머리 터지는 줄
        for k in range(n+1, min(n+1+5, len(route_list[i])+1)):
            # print(tuple(route_list[i][n:k]))
            # 하나일 때랑 여러개 일 때 들어가는 [] 계층이 다름
            if len(route_list[i][n:k])==1: 
                log_ngram_df.loc[route_list[i][n:k]]+=1
            else: 
                try: 
                    log_ngram_df.loc[[tuple(route_list[i][n:k])]]+=1
                except KeyError:
                    continue
log_ngram_df.head()

100%|██████████| 3460762/3460762 [1:05:21<00:00, 882.49it/s] 


,count
StartLoanApply,1694042
UseLoanManage,1423937
UsePrepayCalc,5784
UseDSRCalc,3660
GetCreditInfo,2113765


In [402]:
log_ngram_df.to_csv("./log_ngram_df.csv", index=False)

In [404]:
log_ngram_df.head(20)

,count
StartLoanApply,1694042
UseLoanManage,1423937
UsePrepayCalc,5784
UseDSRCalc,3660
GetCreditInfo,2113765
"(StartLoanApply, UseLoanManage)",138206
"(StartLoanApply, UsePrepayCalc)",6
"(StartLoanApply, UseDSRCalc)",6
"(StartLoanApply, GetCreditInfo)",175437
"(UseLoanManage, StartLoanApply)",127845


## (2) 일시 별로 추가

### 1. 사용자 관련
3) DAU, WAU, MAU
- 일간, 주간, 월간 순사용자
4) 시간대별 방문자 수
- 시간, 일, 주, 월, 휴일 중 사용자 수
5) 사용자별 방문 수 = 세션 수
6) 사용자당 방문 수 = 세션 수/사용자 수

### 2. 화면 - 유저 기준

### 3. 경로 - 스크린 기준

### 4. 액션 - 유저 기준

### 5. 그 외
..